### **Myanmar Case Study** – Projecting Reservoir Emissions Under Construction Date Uncertainty  
#### **Tomasz Janus** – *Manchester, UK, 11/06/2025*

This study explores potential trajectories of biogenic greenhouse gas emissions produced in hydroelectric, multipurpose and irrigation reservoirs in Myanmar under uncertainty about their construction dates. It uses known construction dates of existing reservoirs and information contained in existing hydropower expansion plans to estimate plausible construction timelines for reservoirs with missing information about their construction dates.

Given the limited availability of information about construction dates  of some reservoirs - particularly irrigation reservoirs and planned hydropower projects - we employ random sampling to generate multiple plausible scenarios of historical and future reservoir developments. For each scenario, we simulate the temporal evolution of biogenic emissions, allowing us to assess the likely magnitude and timing of the emission peak and cumulative emissions between 1965 and 2035 / 2040 (two planning horizons).

Reservoirs in Myanmar are categorized into three primary types: *hydroelectric*, *multipurpose*, and *irrigation*. The analysis excludes other types of reservoirs such as *water supply* and *flood control*. Multipurpose reservoirs are assumed to be primarily hydroelectric, with secondary uses including irrigation and potable water supply.

Known construction dates are available for most of the existing hydroelectric and multipurpose reservoirs. However, dates are missing for many irrigation reservoirs and for most planned hydroelectric and multipurpose dams. To address this, we apply the following methodology:

1. **Existing Irrigation Reservoirs:**  
   We calculate a histogram of construction dates using those irrigation reservoirs for which dates are available. This histogram reflects historical trends in irrigation infrastructure investment. We then derive a kernel density estimate (KDE) from which we randomly sample plausible construction years for built irrigation reservoirs with missing construction dates.

2. **Planned Hydroelectric and Multipurpose Dams:**  
   These reservoirs tend to be better documented, however construction dates of planned projects are often not known. In such cases, we sample construction dates uniformly between the present year (2025) and a plausible horizon for project completion (e.g., 2035 or 2040).

Emission trajectories are computed using *Re-emission* software. The results are visualized as ensemble plots, showing a range of potential emission pathways along with the median trajectory, disaggregated by reservoir type (irrigation, multipurpose, and hydroelectric).

#### Period	Activity Highlights - Literature
* 1988–1995 - approx. 97 dams/reservoirs built—or nearly a hundred—initiated/commissioned.
* Mid-1990s–early 2000s	- Bulk of large dams commissioned into service (e.g., Thaphanseik 2002; Paunglin 2004).
* By ~2010 - Total irrigation-dam count exceeded 200.

A breakdown by individual year (e.g., how many were built in 1990 vs. 1995) is impossible to establish as that data isn’t available in public sources. The planning and commissioning clusters clearly centered on 1988–2002.

#### NOTE:
Here, we sample **1000** possible futures such that we have a statistically relevant sample. On most modern computers, the simulation should execute within minutes. However, on older computers the code may run slowly. For testing purposes, in order to speed up the execution, **reduce the sample size** (e.g. down to 100 samples) and/or **increase number of processes** in concurrent futures. Currently the number is set to 4 as this is the number of CPUs available on the machine on which the calculations have been performed.



In [ ]:
from typing import List, NamedTuple, Any, Dict, TypeAlias, Union, Optional, Tuple
from concurrent.futures import ProcessPoolExecutor, as_completed
from itertools import islice
from pathlib import Path
from dataclasses import dataclass, field
from math import floor
import pandas as pd
import geopandas as gpd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KernelDensity
import json
import rich
import pickle
import reemission
from reemission.model import EmissionModel
from reemission.input import Inputs
from reemission.emission_profile import \
    EmissionProfile, EmissionProfileSeries, ureg, Emission

class KDEOutput(NamedTuple):
    x_d: np.ndarray
    density: np.ndarray
    kde: KernelDensity
    
ReservoirName: TypeAlias = str
ScenarioName: TypeAlias = str

dam_db = Path('../data/dams_data/mya_dams.geojson') # Dam information database
input_file = Path('../data/myanmar_inputs.json') # Input data file
construction_years = Path('../data/reservoir_data_with_years.csv') # Reservoir tabular data with construction years
selected_columns = [
    'Project Name', 'Dam Type', 'Function', 'Run-of-River or Storage',
    'COD Year']
gdf_mya = gpd.read_file(dam_db)\
    .filter(items=selected_columns)\
    .query("`Run-of-River or Storage` != 'RoR'")
    
# If roughly 97 dams were built between 1988 and 1995 (see the Introduction at the top of this notebook),
# we can estimate the average number of dams built per year
average_dams_per_year = floor(97 / (1995 - 1988 + 1))
print(f"Average dams built per year: {average_dams_per_year}")

### Function declarations

In [ ]:
def head(data: Dict[str, Any], n: int = 5) -> None:
    """Print the first n items of a dictionary."""
    rich.print(dict(islice(data.items(), 5)))  # Print the first 5 key-value pairs

def save_dict_to_json(data: Dict[str, Any], json_path: str, indent: int = 2) -> None:
    """Save a dictionary to a JSON file."""
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=indent, ensure_ascii=False)

def add_average_counts(df_counts, start_year, end_year, average_count):
    # Make a copy and ensure index is integer type (years)
    df = df_counts.copy()
    df.index = df.index.astype(int)
    # Ensure the column is named 'Count'
    if df.columns[0] != 'Count':
        df.columns = ['Count']
    years = list(range(int(start_year), int(end_year) + 1))
    for year in years:
        if year in df.index:
            df.at[year, 'Count'] += average_count
        else:
            df.loc[year] = average_count
    # Sort by year (index)
    df = df.sort_index()
    return df

def add_counts_from_dict(df_counts: pd.DataFrame, year_count_dict: Dict) -> pd.DataFrame:
    """
    Adds or updates counts in df_counts using a dictionary of {year: count}.
    If the year exists, adds the value; if not, creates a new row.
    """
    df = df_counts.copy()
    df.index = df.index.astype(int)
    if df.columns[0] != 'Count':
        df.columns = ['Count']
    for year, count in year_count_dict.items():
        year = int(year)
        if year in df.index:
            df.at[year, 'Count'] += count
        else:
            df.loc[year] = count
    df = df.sort_index()
    return df

def calculate_kde(count_data: pd.DataFrame, bandwidth=1.0) -> KDEOutput:
    """Calculate Kernel Density Estimation (KDE) for the given count data."""
    counts = count_data['Count']
    years = count_data.index.astype(int)
    # Prepare data for KDE: repeat years according to their count
    kde_data = np.repeat(years, counts.astype(int))
    kde_data = np.array(kde_data)[:, np.newaxis]
    kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(kde_data)
    x_d = np.linspace(years.min(), years.max(), 200)[:, np.newaxis]
    log_dens = kde.score_samples(x_d)
    return KDEOutput(x_d=x_d, density=np.exp(log_dens), kde=kde)
    
def sample_years_kernel(
        kde: KernelDensity,
        n_samples: int, 
        random_state=None):
    """
    Draw n_samples construction years either from an empirical histogram
    or from a pre-fitted KernelDensity estimator.

    Parameters
    ----------
    kde : sklearn.neighbors.KernelDensity
        If provided, samples are drawn from this KDE; otherwise falls back to empirical PMF.
    n_samples : int
        Number of unknown dams to assign years to.
    random_state : int or numpy.random.Generator, optional
        Seed or RNG for reproducibility.

    Returns
    -------
    numpy.ndarray
        Array of length n_samples of sampled years (dtype=int).
    """
    cont = kde.sample(n_samples, random_state=random_state)
    years = np.rint(cont.flatten()).astype(int)
    return years

def sample_years_uniform(
        min_year: int, max_year: int, n_samples: int, random_state=None):
    """
    Draws n_samples years uniformly at random between min(years) and max(years), inclusive.
    Returns a numpy array of sampled years (dtype=int).
    """
    rng = np.random.default_rng(random_state)
    sampled_years = rng.integers(min_year, max_year + 1, size=n_samples)
    return sampled_years

def histogram_plot(count_data: pd.DataFrame, title: str, bandwidth: float = 1.0) -> None:
    """ """
    counts = count_data['Count']
    years = count_data.index.astype(int)
    # Bar plot: years vs counts
    plt.figure(figsize=(10, 5))
    plt.bar(years, counts, color='skyblue', edgecolor='black', label='Counts per Year')
    # KDE (Gaussian)
    # Only plot KDE if there are at least 2 data points
    if len(counts) > 1:
        # Prepare data for KDE: repeat years according to their count
        kde_data = np.repeat(years, counts.astype(int))
        kde_data = np.array(kde_data)[:, np.newaxis]

        kde = KernelDensity(kernel='gaussian', bandwidth=bandwidth).fit(kde_data)
        x_d = np.linspace(years.min(), years.max(), 200)[:, np.newaxis]
        log_dens = kde.score_samples(x_d)
        bin_width = 1  # Each year is a bin
        scale = len(kde_data) * bin_width
        plt.plot(x_d[:, 0], np.exp(log_dens) * scale, color='red', lw=2, label='Gaussian KDE')
        # The scaling factor (/10) is to make the KDE visually comparable to the bar heights; adjust as needed.
    plt.title(title)
    plt.xlabel('Year')
    plt.ylabel('Count')
    plt.grid(axis='y')
    plt.legend()
    plt.show()
        
def histogram_plot_simple(years: List[int], title: str = "") -> None:
    """ """
    plt.figure(figsize=(10, 5))
    plt.hist(
        years, bins=range(min(years), max(years) + 2), color='skyblue', 
        edgecolor='black')
    plt.title(title)
    plt.xlabel('Year')
    plt.ylabel('Frequency')
    plt.grid(axis='y')
    plt.show()

### Generating Probability Distribution from Data to Impute Missing Irrigation Reservoir Construction Years

In [ ]:
%matplotlib inline
# Find known construction years for irrigation reservoirs from the geojson dam database
df_counts_irr = gdf_mya\
    .query("Function == 'IRR'")[["COD Year"]]\
    .value_counts()\
    .sort_index()\
    .rename_axis('Year')\
    .reset_index(name='Count')\
    .set_index('Year')
# Add average number of dams built per year between 1988 and 1995 from data (mentioned in the Introduction)
df_counts_irr = add_average_counts(
    df_counts_irr, start_year=1988, end_year=1995, average_count=average_dams_per_year)
# Add average number of dams built per year between 1997 and 2010
df_counts_irr = add_average_counts(df_counts_irr, 1997, 2010, 1)
# Add dam counts from a manually constructed dictionary between years 1979 and 1988
year_count_dict = {
    1979: 1, 
    1980: 1, 
    1981: 2,
    1982: 3,
    1983: 3,
    1984: 4,
    1985: 5,
    1986: 6,
    1987: 6,
    1988: 6
}
kde_bandwidth = 2.0
df_counts_irr = add_counts_from_dict(df_counts_irr, year_count_dict)
kde_irr = calculate_kde(df_counts_irr, bandwidth=kde_bandwidth)
histogram_plot(
    df_counts_irr,
    title='Histogram of irrigation reservoir construction dates',
    bandwidth=kde_bandwidth)

### Generating multiple construction scenarios for hydroelectric, multipurpose and irrigation reservoirs

In [ ]:
@dataclass
class Scenario:
    """ """
    data: Dict[ScenarioName, Dict[ReservoirName, float]]
    years_irr: List[int]
    years_nonirr: List[int]

@dataclass
class ScenarioGenerator:
    """ """
    input_file: str | Path
    tabulardata_file: str | Path
    irrigation_years_kde: KDEOutput
    number_scenarios: int = 1_0000
    input_data: Dict[str, Any] = field(init=False)
    tabulardata_df: pd.DataFrame = field(init=False)
    
    @staticmethod
    def load_json_to_dict(json_path: str | Path) -> Dict[str, Any]:
        """Load a JSON file and return its contents as a dictionary."""
        json_path = Path(json_path)
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    
    @staticmethod
    def load_csv_to_df(csv_path: str | Path, **kwargs) -> pd.DataFrame:
        """
        Load a CSV file and return its contents as a pandas DataFrame.
        Additional keyword arguments are passed to pd.read_csv.
        """
        csv_path = Path(csv_path)
        return pd.read_csv(csv_path, **kwargs)

    def __post_init__(self) -> None:
        """ """
        self.input_data = self.load_json_to_dict(self.input_file)
        self.tabulardata_df = self.load_csv_to_df(self.tabulardata_file)
        reservoir_names_csv = list(self.tabulardata_df['name'])
        reservoir_names_json = list(self.input_data.keys())
        # Check if both data structures contain the same reservoirs
        missing_in_json = set(reservoir_names_csv) - set(reservoir_names_json)
        missing_in_csv = set(reservoir_names_json) - set(reservoir_names_csv)
        if missing_in_json or missing_in_csv:
            raise ValueError(
                "Reservoir names mismatch between JSON and CSV files.",
                f"Reservoirs in CSV but missing in JSON: {missing_in_json}",
                f"Reservoirs in JSON but missing in CSV: {missing_in_csv}")
            
    def get_projections(self, random_state: int = 42) -> Scenario:
        # Construct a mapping between reseroir names and their construction years for reservoirs with known construction years
        reservoir_years_known = {
            row['name']: int(row['construction_year'])
            for _, row in self.tabulardata_df.iterrows()
            if pd.notna(row['construction_year']) and row['construction_year'] > 0
        }
        irr_reservoirs = self.tabulardata_df.query("type == 'irrigation'")['name'].tolist()
        nonirr_reservoirs = self.tabulardata_df.query("type != 'irrigation'")['name'].tolist()
        irr_reservoirs_unknown = list(set(irr_reservoirs) - set(reservoir_years_known.keys()))
        nonirr_reservoirs_unknown = list(set(nonirr_reservoirs) - set(reservoir_years_known.keys()))
        # Generate years for irrigation reservoir with unknown construction years
        num_irr_unknown = len(irr_reservoirs_unknown)
        num_nonirr_unknown = len(nonirr_reservoirs_unknown)
        irr_years_rnd = np.array(sample_years_kernel(
            kde=self.irrigation_years_kde.kde,
            n_samples=num_irr_unknown * self.number_scenarios,
            random_state=random_state))
        nonirr_years_rnd = np.array(sample_years_uniform(
            min_year=2025, max_year=2045,
            n_samples=num_nonirr_unknown * self.number_scenarios,
            random_state=random_state))
        # Create the scenario data
        chunks_irr = irr_years_rnd.reshape(self.number_scenarios, -1)  # shape: (number_scenarios, num_irr_unknown)
        chunks_nonirr = nonirr_years_rnd.reshape(self.number_scenarios, -1)  # shape: (number_scenarios, num_irr_unknown)   
        scenario_dicts_irr = [dict(zip(irr_reservoirs_unknown, map(int, chunk))) for chunk in chunks_irr]
        scenario_dicts_nonirr = [dict(zip(nonirr_reservoirs_unknown, map(int, chunk))) for chunk in chunks_nonirr]
        scenario_data = {
            sc_index: reservoir_years_known | scenario_dicts_irr[sc_index] | scenario_dicts_nonirr[sc_index]
            for sc_index in range(self.number_scenarios)
        }
        #return scenario_data
        return Scenario(
            data=scenario_data,
            years_irr=irr_years_rnd,
            years_nonirr=nonirr_years_rnd
        )

In [ ]:
# Generate scenarios using the defined ScenarioGenerator class
generator = ScenarioGenerator(
    input_file=input_file, 
    tabulardata_file=construction_years,
    irrigation_years_kde=kde_irr)
scenario = generator.get_projections()

### Check probability distributions of the generated construction years
#### -- They should visually match the distributions they were sampled from --

In [ ]:
histogram_plot_simple(
    scenario.years_irr.flatten(), 
    title='Histogram of sampled irrigation reservoir construction years')

In [ ]:
histogram_plot_simple(
    scenario.years_nonirr.flatten(), 
    title='Histogram of sampled hydroelectric and multipurpose reservoir construction years')

### Calculate emissions and generate emission profiles for each scenario

In [ ]:
# Simulate emissions with ReEmission
input_data = Inputs.fromfile(input_file) 
model = EmissionModel(inputs=input_data, p_model='g-res')
model.calculate()
simulation_outputs = model.outputs
# Calculate and save emission profiles
profiles_per_scenario: Dict[ReservoirName, EmissionProfileSeries] = dict()
year_vector = input_data.inputs[list(input_data.inputs.keys())[0]].data['year_vector']
# Define units for the area and aerial emissions represented in emission profiles
unit_area = ureg('km**2')
unit_profile = ureg('g_CO2e_per_metre2_per_year') # registered unit for emission profiles in emission_profile.py
# Precompute the emission profiles for each reservoir
em_vectors_per_reservoir: Dict[ReservoirName, EmissionProfileSeries] = dict()
res_types: Dict[ReservoirName, str] = dict()
for res_name, res_data in simulation_outputs.items():
    # Get the emission profile for the reservoir
    area = input_data.inputs[res_name].data['reservoir']['area'] * unit_area
    areal_profile_with_unit = \
        np.array(res_data['co2_profile'], dtype=np.float32) * unit_profile + \
        np.array(res_data['ch4_profile'], dtype=np.float32) * unit_profile
    tot_profile = (areal_profile_with_unit * area).to('Mt_CO2e / year')
    # Create an EmissionProfile object
    unit_conv = str(tot_profile.units)
    em = [Emission(value, unit_conv) for value in tot_profile.magnitude]
    em_vectors_per_reservoir[res_name] = em
    res_types[res_name] = input_data.inputs[res_name].data['type']

def build_profile_for_scenario(
    sc_ix: int,
    sc: Dict[str, Union[int, float]],
    year_vector: List[int],
    em_vectors: Dict[str, List[Emission]]) -> Optional[Tuple[int, EmissionProfileSeries]]:
    """ """
    profiles_irr = []
    profiles_nonirr = []
    for res_name, construction_date in sc.items():
        em = em_vectors.get(res_name)
        if em is None or construction_date is None:
            continue
        em_profile = EmissionProfile(values=em, years=year_vector).to_series(construction_date)
        if res_types[res_name] == 'irrigation':
            profiles_irr.append(em_profile)
        else:
            profiles_nonirr.append(em_profile)
    if profiles_irr and profiles_nonirr:
        return sc_ix, EmissionProfileSeries.combine(profiles_irr), EmissionProfileSeries.combine(profiles_nonirr)
    return None

out_path = Path("../outputs/profiles_per_scenario.pkl")
if out_path.exists():
    with out_path.open("rb") as f:
        profiles_per_scenario = pickle.load(f)
else:
    # Container for the results
    profiles_per_scenario: Dict[int, 'EmissionProfileSeries'] = {}
    # Submit parallel tasks
    with ProcessPoolExecutor(max_workers=4) as executor:
        futures = [
            executor.submit(build_profile_for_scenario, sc_ix, sc, year_vector, em_vectors_per_reservoir)
            for sc_ix, sc in scenario.data.items()
        ]
        for future in as_completed(futures):
            result = future.result()
            if result:
                sc_ix, profile_series_irr, profile_series_nonirr = result
                profiles_per_scenario[sc_ix] = {}
                profiles_per_scenario[sc_ix]['irrigation'] = profile_series_irr
                profiles_per_scenario[sc_ix]['nonirrigation'] = profile_series_nonirr
    # dump to disk
    with out_path.open("wb") as f:
        pickle.dump(profiles_per_scenario, f, protocol=pickle.HIGHEST_PROTOCOL)

### Make the emission profile plot for the publication

In [ ]:
from matplotlib.ticker import AutoMinorLocator
def make_publication_plot(
    profiles: Dict[str, Dict[str, EmissionProfileSeries]] = profiles_per_scenario,
    n_series: int | None = 10,
    filename: Path = Path('../outputs/emission_profile.svg')) -> pd.DataFrame:
    """
    Make a publication-ready plot for all emission projections in profiles_per_scenario.
    """
    # 1) Set overall style to serif, tweak font sizes, remove grid, etc.
    plt.rcParams.update({
        'font.family':       'serif',
        'font.serif':        ['Times New Roman', 'Times', 'Palatino'],
        'font.size':         12,
        'axes.titlesize':    14,
        'axes.labelsize':    13,
        'xtick.labelsize':   11,
        'ytick.labelsize':   11,
        'legend.fontsize':   11,
        'axes.linewidth':    1.0,
        'xtick.major.width': 0.8,
        'ytick.major.width': 0.8,
        'xtick.minor.width': 0.6,
        'ytick.minor.width': 0.6,
        'xtick.direction':   'in',
        'ytick.direction':   'in',
        'xtick.bottom':         True,
        'ytick.left':       True,
        'xtick.top':         False,
        'ytick.right':       False,
        'figure.facecolor':  'white',
        'axes.facecolor':    'white',
        'axes.grid':         False,
    })
    
    # 2) Collect the series in lists, plot thin lines as before
    irr_list   = []
    total_list = []
    # Prepare for peak collection
    peak_records = []

    # Create figure
    fig, ax = plt.subplots(figsize=(7.5, 3.75))

    # Loop through each scenario
    for iter, (scenario_id, profiles) in enumerate(profiles.items()):
        if n_series is not None and iter >= n_series:
            break
        # Extract the raw pd.Series of Emission objects
        irr_series = profiles['irrigation'].values       # pd.Series of Emission
        nonirr_series = profiles['nonirrigation'].values # pd.Series of Emission
        # Convert to floats
        irr_vals = irr_series.map(lambda em: em.value)
        nonirr_vals = nonirr_series.map(lambda em: em.value)
        total_vals = irr_vals + nonirr_vals
        
        peak_val  = total_vals.max()
        peak_time = total_vals.idxmax()
        peak_records.append({
            'scenario_id': scenario_id,
            'peak_val':    peak_val,
            'peak_time':   peak_time
        })
        
        # Rename each Series so that concat knows their column name
        irr_list.append(irr_vals.rename(scenario_id))
        total_list.append(total_vals.rename(scenario_id))

        # Plot irrigation alone
        ax.plot(
            irr_vals.index,
            irr_vals.values,
            linewidth=0.3,      # very thin line
            alpha=0.02,          # semi‐transparent
            color='tab:blue'
        )

        # Plot irrigation + nonirrigation
        ax.plot(
            total_vals.index,
            total_vals.values,
            linewidth=0.3,
            alpha=0.02,
            color='tab:orange'
        )
    
    # 3) Build DataFrames & medians
    # Now concatenate all at once (much faster, no fragmentation warnings)
    irr_df   = pd.concat(irr_list,   axis=1)
    total_df = pd.concat(total_list, axis=1)
    # Compute medians
    median_irr = irr_df.median(axis=1)
    median_total = total_df.median(axis=1)
    # Align onto the intersection of their indices so that x, y1, y2 have equal length
    common_index = median_irr.index.intersection(median_total.index)
    median_irr   = median_irr.loc[common_index]
    median_total = median_total.loc[common_index]
        
    # 4) Shade and overplot medians
    ax.fill_between(
        common_index, 
        0, 
        median_irr.values,
        color='lightblue',
        alpha=0.5,
        label='Cumulative emissions from IRR resservoirs'
    )
    # Shade between median irrigation and total
    ax.fill_between(
        common_index,
        median_irr.values,
        median_total.values,
        color='navajowhite',  # light orangeEmiss
        alpha=0.5,
        label='Cumulative emissions from HP and MP resservoirs'
    )
    ax.plot([], [], color='tab:blue', label='IRR')
    ax.plot([], [], color='tab:orange', label='IRR + HP + MP')
    # Plot median lines
    ax.plot(common_index, median_irr.values,
            color='grey', linewidth=2, label='Median emissions')
    ax.plot(common_index, median_total.values,
            color='grey', linewidth=2)
    
    # 5) Compute peaks and add them to the plot
    peaks_df = pd.DataFrame(peak_records)

    sorted_df = peaks_df.sort_values('peak_val').reset_index(drop=True)
    mid_idx = len(sorted_df) // 2
    med_row = sorted_df.iloc[mid_idx]
    
    # Global max‐peak
    max_row = peaks_df.loc[peaks_df['peak_val'].idxmax()]
    # Global min‐peak
    min_row = peaks_df.loc[peaks_df['peak_val'].idxmin()]
    # Mark total max, min, and median peak scenarios
    ax.scatter(max_row['peak_time'], max_row['peak_val'], linewidth=0.4,
               color='tab:orange', marker='o', s=30, zorder=5, alpha=0.6, edgecolor='black', 
               label="Maximum peak emission")
    ax.scatter(min_row['peak_time'], min_row['peak_val'], linewidth=0.4,
               color='tab:blue', marker='o', s=30, zorder=5, alpha=0.6, edgecolor='black', 
               label="Minimum peak emission")
    # NEW: annotate median peak scenario
    ax.scatter(med_row['peak_time'], med_row['peak_val'], linewidth=0.4,
               color='tab:green', marker='o', s=30, zorder=5, alpha=0.6, edgecolor='black', 
               label=("Median peak emission"))

    # 6) Axes formatting
    ax.set_xlim(pd.Timestamp('1960-01-01'), common_index.max())
    ax.set_ylim(bottom=0)
    # Tight ticks, minor ticks on
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    # Labels & title
    ax.set_xlabel('Year')
    ax.set_ylabel('GHG Emissions (Mt CO2e / year)')
    #plt.title('Irrigation vs. Total Emission Profiles (all scenarios)')

    # (Optional) add a legend entry manually if you like
    ax.legend(frameon=False, loc='upper left')
    fig.tight_layout()
    if filename:
        fig.savefig(filename, format='svg', dpi=300)
    plt.show()
    return peaks_df
    
make_publication_plot(n_series=1_000)